# **The eighth in-class-exercise (20 points in total, 10/29/2020)**

The data for this exercise is from the dataset you created from assignment three. Please perform answer the following questions based on your data:

## (1) (10 points) Write a python program to extract the sentiment related terms from the corpus. You may use python package such as polyglot or external lexicon resources in the question. Rank the sentiment related terms by frequency.

In [1]:
# Write your code here
import pandas as pd
import re
import nltk
from nltk import word_tokenize

- The data I collected in Assignment 3 has only titles, which is irrelevant for sentiment analysis. 

- I used Restaurant Review data set from Kaggle.

In [2]:
df = pd.read_table('/content/Restaurant_Reviews.tsv')

In [3]:
df.pop('Liked')

0      1
1      0
2      0
3      1
4      1
      ..
995    0
996    0
997    0
998    0
999    0
Name: Liked, Length: 1000, dtype: int64

In [4]:
df.head()

,Review
0,Wow... Loved this place.
1,Crust is not good.
2,Not tasty and the texture was just nasty.
3,Stopped by during the late May bank holiday of...
4,The selection on the menu was great and so wer...


**DATA** **CLEANING**

In [5]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [6]:
import string
df['Review'] = df['Review'].apply(lambda x: x.strip(string.punctuation))
df['Review'][2]

'Not tasty and the texture was just nasty'

In [7]:
import re
df['Review_punc'] = df['Review'].apply(lambda x: re.sub(r'[^a-zA-Z0-9]+',' ',x))
df['Review_punc'][0]

'Wow Loved this place'

In [8]:
df['Review_lower'] = df['Review_punc'].apply(lambda x: x.lower())
df['Review'][2]

'Not tasty and the texture was just nasty'

In [9]:
stop = ['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', 'too', 'very', 's', 't', 'can', 'will', 'just', 'don', "don't", 'should', "should've", 'now', 'd', 'll', 'm', 'o', 're', 've', 'y', 'ain', 'aren', "aren't", 'couldn', "couldn't", 'didn', "didn't", 'doesn', "doesn't", 'hadn', "hadn't", 'hasn', "hasn't", 'haven', "haven't", 'isn', "isn't", 'ma', 'mightn', "mightn't", 'mustn', "mustn't", 'needn', "needn't", 'shan', "shan't", 'shouldn', "shouldn't", 'wasn', "wasn't", 'weren', "weren't", 'won', "won't", 'wouldn', "wouldn't"]
df['Review_stop'] = df['Review_lower'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))
# test['tweet_without_stopwords'] = test['tweet_without_stopwords'].str.replace(r'\s+', ' ')
df['Review_stop'][2]

'tasty texture nasty'

In [10]:
!pip install polyglot
!pip install pyicu
!pip install pycld2
!pip install morfessor

In [11]:
!polyglot download sentiment2.en

[polyglot_data] Downloading package sentiment2.en to
[polyglot_data]     /root/polyglot_data...
[polyglot_data]   Package sentiment2.en is already up-to-date!


In [12]:
from icu import Locale
import pycld2 as cld2
from polyglot.text import Text
df['Review_stop'].head()

0                                      wow loved place
1                                           crust good
2                                  tasty texture nasty
3    stopped late may bank holiday rick steve recom...
4                          selection menu great prices
Name: Review_stop, dtype: object

In [13]:
df.dropna()
df['Review_stop'] = df['Review_stop'].str.replace('\d+', '')

In [14]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [15]:
df['Reviews_Token'] = df['Review_stop'].apply(lambda x: word_tokenize(x))
tex = [Text (df['Review_stop'][i],hint_language_code='en') for i in range(len(df['Review_stop']))]

In [16]:
tex.remove(Text(""))

In [17]:
from polyglot.detect.base import logger as polyglot_logger
polyglot_logger.setLevel("ERROR")

In [18]:
list_word = [ ]
for i in tex[:1000]:
 for j in i.words:
      list_word.append(j)

In [19]:
from collections import Counter

In [20]:
list_count = Counter(list_word)
len(list_count)
len(list_word)

5486

In [21]:
df_Rank = pd.DataFrame({'words': list (list_count.keys()),'Freq': list (list_count.values()),'polarity': list (i.polarity for i in list_count.keys())})

In [22]:
df_Rank = df_Rank[df_Rank['polarity'] !=0]

In [23]:
df_Rank = df_Rank.sort_values(by=['Freq'], ascending=False)

In [24]:
import numpy as np
Index = np.arange(1,len(df_Rank)+1)

In [25]:
df_Rank.insert(0,'Rank',Index)

In [26]:
Index = np.arange(1,len(df_Rank)+1)
df_Rank.reset_index(drop=True)

,Rank,words,Freq,polarity
0,1,good,95,1
1,2,great,70,1
2,3,like,46,1
3,4,best,30,1
4,5,friendly,27,1
...,...,...,...,...
327,328,screwed,1,-1
328,329,frustrated,1,-1
329,330,petty,1,-1
330,331,sore,1,-1


In [27]:
pol_freq = Counter(list (df_Rank['polarity']))

In [28]:
pol_freq

Counter({-1: 165, 1: 167})

As we can see the restaurant reviews has equal amount of negative(-1) as well as positive(1) reviews 

## (2) (10 points) Compare the performance of the following tools in sentiment identification: TextBlob (https://textblob.readthedocs.io/en/dev/), VADER (https://github.com/cjhutto/vaderSentiment), TFIDF-based Support Vector Machine (SVM) (Split your data into training and testing data). Take your own annotation as the standard answers. 

Reference code: https://towardsdatascience.com/fine-grained-sentiment-analysis-in-python-part-1-2697bb111ed4

In [29]:
# Write your code here





# Your analysis here

